# Лаболаторная работа №2. Код Хемминга и контрольные суммы

Машуров Владимир БПМ-19-3

## Код

Библиотеки и переменные

In [257]:
import zlib
import copy
import random

from tqdm import tqdm
import numpy as np

WORD_LEN = 82
RESERVED = 8
MSG_LEN = WORD_LEN - RESERVED

Функции для перевода между типами и представлениями данных

In [258]:
def text_to_bits(text, encoding="utf-8"):
    bits = bin(int.from_bytes(text.encode(encoding), "big"))[2:]
    return bits.zfill(8 * ((len(bits) + 7) // 8))
 
 
def text_from_bits(bits, encoding="utf-8"):
    n = int(bits, 2)
    return n.to_bytes((n.bit_length() + 7) // 8, "big").decode(encoding) or "\0"

def str_generator(str: str, word_len: int=75):
        i = 0
        while i * word_len < len(str):
            yield str[i * word_len: word_len * (i+1)]
            i += 1

def str2bytes(word: str):
    b = 0
    word = word[::-1]
    for i, sign in enumerate(word):
        if int(sign) == 1:
            b += 2 ** i
    return b.to_bytes(16, 'big')

Класс сообщения, где и реализован алгоритм

In [259]:
class Message():
    def __init__(self, msg: str) -> None:
        self.msg = msg
        self.msg_encoded = text_to_bits(self.msg)
        self.words = [i for i in str_generator(self.msg_encoded, word_len=MSG_LEN)]
        self.fixed_words = 0
        self.unfixed_words = 0

    def set_tale(self, word: str) -> str:
        sum1 = 0
        sum2 = 0
        sum4 = 0
        sum8 = 0
        sum16 = 0
        sum32 = 0
        sum64 = 0
        checksum = 0

        for i in range(len(word)):
            checksum += int(word[i])
            if i in self.resp1:
                sum1 += int(word[i])
            if i in self.resp2:
                sum2 += int(word[i])
            if i in self.resp4:
                sum4 += int(word[i])
            if i in self.resp8:
                sum8 += int(word[i])
            if i in self.resp16:
                sum16 += int(word[i])
            if i in self.resp32:
                sum32 += int(word[i])
            if i in self.resp64:
                sum64 += int(word[i])

        word = list(word)
        word.append(str(sum1 % 2))
        word.append(str(sum2 % 2))
        word.append(str(sum4 % 2))
        word.append(str(sum8 % 2))
        word.append(str(sum16 % 2))
        word.append(str(sum32 % 2))
        word.append(str(sum64 % 2))
        word.append(str(checksum % 2))
        word = "".join(word)
        return word

    def encode(self) -> None:
        self.resp1 = np.array([np.array(range(1))   + i      for i in range(100) if i % 2 == 1]).reshape(-1) - 1
        self.resp2 = np.array([np.array(range(2))   + i * 2  for i in range(100) if i % 2 == 1]).reshape(-1) - 1
        self.resp4 = np.array([np.array(range(4))   + i * 4  for i in range(100) if i % 2 == 1]).reshape(-1) - 1
        self.resp8 = np.array([np.array(range(8))   + i * 8  for i in range(100) if i % 2 == 1]).reshape(-1) - 1
        self.resp16 = np.array([np.array(range(16)) + i * 16 for i in range(100) if i % 2 == 1]).reshape(-1) - 1
        self.resp32 = np.array([np.array(range(32)) + i * 32 for i in range(100) if i % 2 == 1]).reshape(-1) - 1
        self.resp64 = np.array([np.array(range(64)) + i * 64 for i in range(100) if i % 2 == 1]).reshape(-1) - 1

        for word_id, word in enumerate(self.words):
            word = self.set_tale(word)
            self.words[word_id] = word

    @staticmethod
    def get_tale(word: str):
        tale  = word[-RESERVED:]
        sum1     = tale[0]
        sum2     = tale[1]
        sum4     = tale[2]
        sum8     = tale[3]
        sum16    = tale[4]
        sum32    = tale[5]
        sum64   =  tale[6]
        checksum = tale[7]
        return sum1, sum2, sum4, sum8, sum16, sum32, sum64, checksum


    def detect_errors(self, new_msg):

        log =  "| word n | crc | checksum | err_index | multiple | \n"
        log += "|---|---|---|---|---| \n"

        for i, word in tqdm(enumerate(new_msg.words)):

            word = self.set_tale(word=word[:-RESERVED])

            crc_new = zlib.crc32(str2bytes(word))
            crc_orig = zlib.crc32(str2bytes(self.words[i]))

            log += f"| {i} | {crc_orig == crc_new} |"
            
            err_occlusions = []
            sum1, sum2, sum4, sum8, sum16, sum32, sum64, checksum = self.get_tale(word)
            sum1_self, sum2_self, sum4_self, sum8_self, sum16_self, sum32_self, sum64_self, checksum_self = self.get_tale(self.words[i])

            log += f" {checksum_self == checksum} |"

            if sum1 != sum1_self:
                err_occlusions.append(1)
            if sum2 != sum2_self:
                err_occlusions.append(2)
            if sum4 != sum4_self:
                err_occlusions.append(4)
            if sum8 != sum8_self:
                err_occlusions.append(8)
            if sum16 != sum16_self:
                err_occlusions.append(16)
            if sum32 != sum32_self:
                err_occlusions.append(32)
            if sum64 != sum64_self:
                err_occlusions.append(64)

            err_index = sum(err_occlusions)
            log += f" {err_index} |"
            err_index -= 1

            if err_index > -1: 
                try:
                    fixed_word = copy.copy(word)
                    fixed_word = list(fixed_word)
                    fixed_word[err_index] = self.words[i][err_index]
                    fixed_word = "".join(fixed_word)
                    fixed_word = fixed_word[:-RESERVED]
                    fixed_word = self.set_tale(fixed_word)

                    if zlib.crc32(str2bytes(fixed_word)) != zlib.crc32(str2bytes(self.words[i])):
                        log += " True | \n"
                        new_msg.unfixed_words += 1
                    else:
                        new_msg.fixed_words += 1
                        log += " False | \n"
                except:
                    log += " True | \n"
                    new_msg.unfixed_words += 1
            else:
                log += " False | \n"
            
        return log

## Небольшой пример работы

In [260]:
foo = Message("tree, сегодня christmas!")
foo.encode()

Испортим значение в 4-м бите

In [261]:
new_msg = copy.copy(foo.msg_encoded)
new_msg = list(new_msg)
new_msg[3] = '0'
new_msg[100] = '1'
new_msg[101] = '1'
new_msg = "".join(new_msg)
new_msg = text_from_bits(new_msg)
new_msg = Message(new_msg)
new_msg.encode()

Заметим, что сообщение изменилось. t стало d

In [262]:
new_msg.msg

'dree, сегܾдня christmas!'

Детектируем ошибки и логируем

In [263]:
log = foo.detect_errors(new_msg=new_msg)
with open("./logs", "w") as log_file:
    log_file.write(log)

4it [00:00, 426.77it/s]


Мы получили следующую таблицу, где 

- word n -- номер пакета;
- crc -- совпадают ли crc32 контрольные суммы сообщений;
- checksum -- совпадает ли контрольная сумма, в последнем бите слова;
- err_index -- вычисленный индекс поврежденного бита. 0 значит, что ошибок не обнаружено;
- multiple -- если ошибка более, чем в одном бите, то примет значение True.


| word n | crc | checksum | err_index | multiple | 
|---|---|---|---|---| 
| 0 | False | False | 4 | False | 
| 1 | False | True | 7 | True | 
| 2 | True | True | 0 | False | 
| 3 | True | True | 0 | False | 


In [264]:
print(f"Исправленных слов {new_msg.fixed_words}")
print(f"Не исправленных слов {new_msg.unfixed_words}")

Исправленных слов 1
Не исправленных слов 1


# Пример со статьёй из Википедии

Был выбран следующий текст:
```
Тра́вля (бу́ллинг — англ. bullying в переводе запугивание) — агрессивное преследование и издевательство над одним из членов коллектива (особенно коллектива школьников и студентов, но также и коллег) со стороны другого, но также часто группы лиц, не обязательно из одного формального или признаваемого другими коллектива. Травлю организует один агрессор (хулиган), иногда с сообщниками, а большинство остаются свидетелями. При травле жертва оказывается не в состоянии защитить себя от нападок, таким образом, травля отличается от конфликта, где силы сторон примерно равны. Травля может быть и в физической, и в психологической форме. Проявляется во всех возрастных и социальных группах. В сложных случаях может принять некоторые черты групповой преступности. В качестве особой формы травли выделяют групповую травлю («травля толпы»), большинством или всеми членами коллектива (микросообщества), часто начальником, работодателем (жарг. «моббинг»). Как проявления травли специалисты расценивают оскорбления, угрозы, физическую агрессию, постоянную негативную оценку жертвы и её деятельности, отказ в доверии и делегировании полномочий и так далее. Есть случаи, когда жертвами травли становились учителя[2]. Объединяет всех жертв одна черта: чаще всего объектами издевательств становятся дети и подростки, обладающие повышенной чувствительностью, показывающие свою «слабость» (страх, обиду или злость). Их реакция соответствует ожиданиям агрессоров, рождая искомое ощущение превосходства[4]. Мальчики чаще являются жертвами и инициаторами школьной травли. Методы травли отличаются в зависимости от пола жертвы: мальчиков чаще бьют, а про девочек одноклассницы, как правило, распускают порочащие слухи[2][5]. По мнению педагога-психолога Людмилы Петрановской, «это иллюзия, что для <роли козла отпущения> надо быть каким-то особо ненормальным. В общем и целом причина травли — не в особенностях жертвы, а в особенностях группы. Один и тот же ребёнок может быть изгоем в одной группе и своим в другой. Или перестать быть изгоем в той же самой за короткий срок, скажем, после смены классного руководителя»[6]. Большую роль в школьной травле играет гендерный надзор — действия, направленные на принуждение к следованию предписанным гендерным нормам (для девочек — быть женственными, для мальчиков — быть мужественными). Во многом в связи с тем, такие действия не носят явно выраженного антисоциального характера, а напротив, связаны с доминирующими в обществе установками, «травящие» во многих случаях не получают реального осуждения со стороны взрослых. Гендерный надзор тесно связан с гетеронормативностью. Травля мальчиков, вне зависимости от их реальной сексуальной ориентации, как правило, активно сопровождается гомофобными оскорблениями. Среди оскорблений девочек преобладают слатшейминговые[7][8].
```

In [265]:
wiki_msg = "Тра́вля (бу́ллинг — англ. bullying в переводе запугивание) — агрессивное преследование и издевательство над одним из членов коллектива (особенно коллектива школьников и студентов, но также и коллег) со стороны другого, но также часто группы лиц, не обязательно из одного формального или признаваемого другими коллектива. Травлю организует один агрессор (хулиган), иногда с сообщниками, а большинство остаются свидетелями. При травле жертва оказывается не в состоянии защитить себя от нападок, таким образом, травля отличается от конфликта, где силы сторон примерно равны. Травля может быть и в физической, и в психологической форме. Проявляется во всех возрастных и социальных группах. В сложных случаях может принять некоторые черты групповой преступности. В качестве особой формы травли выделяют групповую травлю («травля толпы»), большинством или всеми членами коллектива (микросообщества), часто начальником, работодателем (жарг. «моббинг»). Как проявления травли специалисты расценивают оскорбления, угрозы, физическую агрессию, постоянную негативную оценку жертвы и её деятельности, отказ в доверии и делегировании полномочий и так далее. Есть случаи, когда жертвами травли становились учителя[2]. Объединяет всех жертв одна черта: чаще всего объектами издевательств становятся дети и подростки, обладающие повышенной чувствительностью, показывающие свою «слабость» (страх, обиду или злость). Их реакция соответствует ожиданиям агрессоров, рождая искомое ощущение превосходства[4]. Мальчики чаще являются жертвами и инициаторами школьной травли. Методы травли отличаются в зависимости от пола жертвы: мальчиков чаще бьют, а про девочек одноклассницы, как правило, распускают порочащие слухи[2][5]. По мнению педагога-психолога Людмилы Петрановской, «это иллюзия, что для <роли козла отпущения> надо быть каким-то особо ненормальным. В общем и целом причина травли — не в особенностях жертвы, а в особенностях группы. Один и тот же ребёнок может быть изгоем в одной группе и своим в другой. Или перестать быть изгоем в той же самой за короткий срок, скажем, после смены классного руководителя»[6]. Большую роль в школьной травле играет гендерный надзор — действия, направленные на принуждение к следованию предписанным гендерным нормам (для девочек — быть женственными, для мальчиков — быть мужественными). Во многом в связи с тем, такие действия не носят явно выраженного антисоциального характера, а напротив, связаны с доминирующими в обществе установками, «травящие» во многих случаях не получают реального осуждения со стороны взрослых. Гендерный надзор тесно связан с гетеронормативностью. Травля мальчиков, вне зависимости от их реальной сексуальной ориентации, как правило, активно сопровождается гомофобными оскорблениями. Среди оскорблений девочек преобладают слатшейминговые[7][8]."
wiki_msg = Message(wiki_msg)
wiki_msg.encode()

### Без ошибок

В данном случае, разумеется, не было исправлено ни одного слова.

In [266]:
new_msg = copy.copy(wiki_msg.msg_encoded)
new_msg = text_from_bits(new_msg)
new_msg = Message(new_msg)
new_msg.encode()

log = wiki_msg.detect_errors(new_msg=new_msg)
with open("./logs_no_errs", "w") as log_file:
    log_file.write(log)

554it [00:00, 906.26it/s]


In [267]:
print(f"Исправленных слов {new_msg.fixed_words} из {new_msg.fixed_words + new_msg.unfixed_words}")
print(f"Не исправленных слов {new_msg.unfixed_words} из {new_msg.fixed_words + new_msg.unfixed_words}")
print(f"Исправленных ошибок {new_msg.fixed_words} из 0")

Исправленных слов 0 из 0
Не исправленных слов 0 из 0
Исправленных ошибок 0 из 0


### Не более одной ошибки на слово

Зашумляем сигнал, не более одной ошибки на слово.

In [268]:
good_msg = copy.deepcopy(wiki_msg.words)
new_msg =  copy.deepcopy(wiki_msg)
err_count = 0

for i in range(len(new_msg.words)):
    if random.randint(0, 1) == 0:
        err_count += 1
        rand_idx = random.randint(0, len(new_msg.words[i])-RESERVED-1)

        new_msg.words[i] = list(new_msg.words[i])

        if good_msg[i][rand_idx] == '0':
            new_msg.words[i][rand_idx] = '1'
        else:
            new_msg.words[i][rand_idx] = '0'

        new_msg.words[i] = "".join(new_msg.words[i])

Есть возможность логировать таблицу Markdown

In [269]:
log = wiki_msg.detect_errors(new_msg=new_msg)
with open("./logs_one_err", "w") as log_file:
    log_file.write(log)

554it [00:00, 623.55it/s]


Статистика по испревленным словам

In [270]:
print(f"Исправленных слов {new_msg.fixed_words} из {new_msg.fixed_words + new_msg.unfixed_words}")
print(f"Не исправленных слов {new_msg.unfixed_words} из {new_msg.fixed_words + new_msg.unfixed_words}")
print(f"Исправленных ошибок {new_msg.fixed_words} из {err_count}")

Исправленных слов 276 из 276
Не исправленных слов 0 из 276
Исправленных ошибок 276 из 276


### Более одной ошибкии на слово

Зашумляем сигнал, не более пяти ошибок на слово

In [271]:
good_msg = copy.deepcopy(wiki_msg.words)
new_msg =  copy.deepcopy(wiki_msg)
err_count = 0

for i in range(len(new_msg.words)):
    added_to_word_errs = set()
    for _ in range(random.randint(0, 5)):
        rand_idx = random.randint(0, len(new_msg.words[i])-RESERVED-1)
        added_to_word_errs.add(rand_idx)

        new_msg.words[i] = list(new_msg.words[i])

        if good_msg[i][rand_idx] == '0':
            new_msg.words[i][rand_idx] = '1'
        else:
            new_msg.words[i][rand_idx] = '0'

        new_msg.words[i] = "".join(new_msg.words[i])
    err_count += len(added_to_word_errs)

Есть возможность логировать таблицу Markdown

In [272]:
log = wiki_msg.detect_errors(new_msg=new_msg)
with open("./logs_multiple_errs", "w") as log_file:
    log_file.write(log)

554it [00:01, 553.76it/s]


Статистика по испревленным словам

In [273]:
print(f"Исправленных слов {new_msg.fixed_words} из {new_msg.fixed_words + new_msg.unfixed_words}")
print(f"Не исправленных слов {new_msg.unfixed_words} из {new_msg.fixed_words + new_msg.unfixed_words}")
print(f"Исправленных ошибок {new_msg.fixed_words} из {err_count}")

Исправленных слов 106 из 463
Не исправленных слов 357 из 463
Исправленных ошибок 106 из 1396
